In [1]:
using Plots
using Random
using Statistics

function generate_population(size, rrange)
    r1, r2 = rrange
    return [rand() * (r2 - r1) + r1 for _ in 1:size]
end

function mutate(x, mutation_rate, rrange)
    r1, r2 = rrange
    if rand() < mutation_rate
        return rand() * (r2 - r1) + r1
    end
    return x
end

function select(population, fitness, number_of_parents)
    scores = [fitness(x) for x in population]
    sorted_idx = sortperm(scores)
    return population[sorted_idx[1:number_of_parents]]
end

# Функция для одноточечного кроссовера через двоичное представление
function binary_single_point_crossover(parent1, parent2, x_range)
    # Преобразуем числа в битовые представления
    p1_bits = bitstring(Float64(parent1))
    p2_bits = bitstring(Float64(parent2))
    
    # Выбираем точку кроссинговера
    crossover_point = rand(1:length(p1_bits))
    
    # Создаем новые битовые строки путем обмена
    c1_bits = p1_bits[1:crossover_point] * p2_bits[crossover_point+1:end]
    c2_bits = p2_bits[1:crossover_point] * p1_bits[crossover_point+1:end]
    
    # Преобразуем обратно в числа
    child1 = parent1
    child2 = parent2
    
    try
        child1 = reinterpret(Float64, parse(UInt64, c1_bits, base=2))
        child2 = reinterpret(Float64, parse(UInt64, c2_bits, base=2))
    catch
        # Если преобразование не удалось, используем оригинальные значения
    end
    
    # Проверка на допустимость значений
    if isnan(child1) || isinf(child1)
        child1 = parent1
    end
    if isnan(child2) || isinf(child2)
        child2 = parent2
    end
    
    # Фиксируем диапазон значений
    child1 = clamp(child1, x_range[1], x_range[2])
    child2 = clamp(child2, x_range[1], x_range[2])
    
    return child1, child2, crossover_point
end

# Обновленная функция кроссовера с выбором метода
function crossover(parents, number_of_children, crossover_type, x_range)
    children = []
    crossover_points = []  # Сохраняем информацию о точках кроссовера
    
    if crossover_type == "average"
        # Среднее значение родителей
        children = [(rand(parents) + rand(parents)) / 2 for _ in 1:number_of_children]
        crossover_points = zeros(Int, number_of_children)  # Нет точек кроссовера
        
    elseif crossover_type == "binary"
        # Одноточечный кроссовер с двоичным представлением
        while length(children) < number_of_children
            # Выбираем двух случайных родителей
            p1, p2 = rand(parents), rand(parents)
            
            # Выполняем кроссовер
            c1, c2, cp = binary_single_point_crossover(p1, p2, x_range)
            
            push!(children, c1)
            push!(crossover_points, cp)
            
            if length(children) < number_of_children
                push!(children, c2)
                push!(crossover_points, cp)
            end
        end
    else
        error("Неизвестный тип кроссовера: $crossover_type")
    end
    
    return children, crossover_points
end

function genetic_algorithm(fitness, generations, population_size, x_range, mutation_rate, crossover_type="average", name="test")
    Random.seed!(123)  # Фиксируем seed для воспроизводимости
    population = generate_population(population_size, x_range)
    best_idx = argmin([fitness(x) for x in population])
    best_solution = population[best_idx]
    best_score = fitness(best_solution)
    
    # История лучших решений
    best_history = [(best_solution, best_score)]
    
    # Подготовка анимации
    anim = Animation()
    x_vals = range(x_range[1], x_range[2], length=200)
    y_vals = fitness.(x_vals)
    y_min, y_max = minimum(y_vals), maximum(y_vals)
    plot_range = (y_min - 0.1*(y_max-y_min), y_max + 0.1*(y_max-y_min))
    
    # Создаем первый кадр
    plt = plot(x_vals, y_vals, 
               color=:blue, linewidth=2, 
               legend=:topright,
               title="Генетический алгоритм - Поколение 0 - $(crossover_type) кроссовер",
               xlims=x_range, ylims=plot_range,
               xlabel="X", ylabel="f(X)",
               size=(800, 600), grid=true)
    
    scatter!(plt, population, fitness.(population), 
             color=:lightgreen, alpha=0.7, markersize=4,
             label="Популяция")
    
    scatter!(plt, [best_solution], [best_score], 
             color=:red, markersize=8, marker=:star,
             label="Лучшая особь: x=$(round(best_solution, digits=3))")
             
    annotate!(plt, x_range[1] + 0.1*(x_range[2]-x_range[1]), 
              plot_range[1] + 0.1*(plot_range[2]-plot_range[1]),
              text("f(x) = $(round(best_score, digits=3))", :left, 10))
    
    frame(anim)

    # Статистика кроссовера
    crossover_stats = []
    
    for generation in 1:generations
        # Отбор родителей
        parents = select(population, fitness, population_size ÷ 2)
        
        # Скрещивание с выбранным методом
        children, crossover_points = crossover(parents, population_size - length(parents), crossover_type, x_range)
        
        # Сохраняем статистику точек кроссовера
        if crossover_type == "binary"
            push!(crossover_stats, crossover_points)
        end
        
        # Мутация
        children = [mutate(c, mutation_rate, x_range) for c in children]
        
        # Новая популяция
        population = vcat(parents, children)
        
        # Поиск лучшего решения
        current_scores = [fitness(x) for x in population]
        current_best_idx = argmin(current_scores)
        current_best = population[current_best_idx]
        current_score = current_scores[current_best_idx]

        # Отладочный вывод
        if generation % 10 == 0
            println("\nПоколение $generation")
            println("  Среднее: ", round(mean(population), digits=2))
            println("  Лучшее: ", round(current_best, digits=3), 
                    " (f = ", round(current_score, digits=3), ")")
            println("  Разнообразие: ", round(std(population), digits=3))
            
            if crossover_type == "binary"
                avg_crossover_point = round(mean(crossover_points), digits=0)
                println("  Средняя точка кроссовера: $avg_crossover_point")
            end
        end
# Обновление лучшего решения
if current_score < best_score
    best_solution = current_best
    best_score = current_score
    push!(best_history, (best_solution, best_score))
    
    println("Поколение $generation: Новое лучшее = ", 
           round(best_solution, digits=4), 
           " (Значение: ", round(best_score, digits=4), ")")
end

# Создаем кадр для анимации
plt = plot(x_vals, y_vals, 
          color=:blue, linewidth=2,
          legend=:topright,
          title="Генетический алгоритм - Поколение $generation",
          xlims=x_range, ylims=plot_range,
          xlabel="X", ylabel="f(X)",
          size=(800, 600), grid=true)

# Отображаем текущую популяцию
scatter!(plt, population, fitness.(population), 
         color=:lightgreen, alpha=0.7, markersize=4,
         label="Популяция")


# Отображаем глобальное лучшее решение
scatter!(plt, [best_solution], [best_score], 
         color=:red, markersize=8,
         label="Лучшее решение: x=$(round(best_solution, digits=3))")

frame(anim)
end

# Сохраняем анимацию
gif(anim, "genetic_algorithm_$(crossover_type)_$(name).gif", fps=5)
println("Анимация сохранена в файл: genetic_algorithm_$(crossover_type)_$(name).gif")

return best_solution
end

# Определение целевой функции
fit(x) = (x+1)*(x+2)*((x+3)^2)
# fit(x) = 5 - 24 * x + 17 * x^2 - 11/3 * x^3 + 1/4 * x^4

# Параметры с выбором типа кроссовера
params = (
generations = 100,
population_size = 50,
x_range = (-4.0, 0.0),
mutation_rate = 0.2,
crossover_type = "binary"  # "average" или "binary"
)

# Запуск алгоритма с выбранным типом кроссовера
@time best = genetic_algorithm(fit, params.generations, params.population_size, 
                     params.x_range, params.mutation_rate, params.crossover_type,"test1")

println("\n\nФинальный результат:")
println("Лучшее решение: x = ", round(best, digits=4), 
", f(x) = ", round(fit(best), digits=4))


Поколение 3: Новое лучшее = -1.3421 (Значение: -0.6186)
Поколение 7: Новое лучшее = -1.3421 (Значение: -0.6186)

Поколение 10
  Среднее: -1.47
  Лучшее: -1.342 (f = -0.619)
  Разнообразие: 0.515
  Средняя точка кроссовера: 34.0
Поколение 12: Новое лучшее = -1.3421 (Значение: -0.6186)
Поколение 13: Новое лучшее = -1.3421 (Значение: -0.6186)
Поколение 14: Новое лучшее = -1.3729 (Значение: -0.6191)
Поколение 18: Новое лучшее = -1.3729 (Значение: -0.6191)

Поколение 20
  Среднее: -1.4
  Лучшее: -1.373 (f = -0.619)
  Разнообразие: 0.252
  Средняя точка кроссовера: 24.0
Поколение 23: Новое лучшее = -1.3729 (Значение: -0.6191)
Поколение 28: Новое лучшее = -1.3729 (Значение: -0.6191)

Поколение 30
  Среднее: -1.4
  Лучшее: -1.373 (f = -0.619)
  Разнообразие: 0.245
  Средняя точка кроссовера: 40.0

Поколение 40
  Среднее: -1.46
  Лучшее: -1.373 (f = -0.619)
  Разнообразие: 0.446
  Средняя точка кроссовера: 24.0
Поколение 49: Новое лучшее = -1.3585 (Значение: -0.6197)

Поколение 50
  Среднее: -1

┌ Info: Saved animation to /Users/d.okutin/genetic_algorithm_binary_test1.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


  8.602787 seconds (5.51 M allocations: 384.935 MiB, 3.50% gc time, 29.03% compilation time: <1% of which was recompilation)


Финальный результат:
Лучшее решение: x = -1.3592, f(x) = -0.6197


In [2]:
# fit(x) = x^4 + 7*x^3 - 15*x^2 - 175*x - 250
fit(x) = 5 - 24 * x + 17 * x^2 - 11/3 * x^3 + 1/4 * x^4

# Параметры с выбором типа кроссовера
params = (
generations = 100,
population_size = 50,
x_range = (-2.0, 9.0),
mutation_rate = 0.2,
crossover_type = "binary"  # "average" или "binary"
)

# Запуск алгоритма с выбранным типом кроссовера
@time best = genetic_algorithm(fit, params.generations, params.population_size, 
                     params.x_range, params.mutation_rate, params.crossover_type, "test2")

println("\n\nФинальный результат:")
println("Лучшее решение: x = ", round(best, digits=4), 
", f(x) = ", round(fit(best), digits=4))

Поколение 3: Новое лучшее = 1.0426 (Значение: -5.4033)
Поколение 4: Новое лучшее = 1.0353 (Значение: -5.4074)

Поколение 10
  Среднее: 1.02
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 0.704
  Средняя точка кроссовера: 34.0
Поколение 11: Новое лучшее = 1.0353 (Значение: -5.4074)
Поколение 12: Новое лучшее = 1.0353 (Значение: -5.4074)
Поколение 16: Новое лучшее = 1.0353 (Значение: -5.4074)

Поколение 20
  Среднее: 1.55
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 1.465
  Средняя точка кроссовера: 24.0

Поколение 30
  Среднее: 1.22
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 0.97
  Средняя точка кроссовера: 40.0

Поколение 40
  Среднее: 1.21
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 1.096
  Средняя точка кроссовера: 24.0

Поколение 50
  Среднее: 1.03
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 0.013
  Средняя точка кроссовера: 36.0

Поколение 60
  Среднее: 1.34
  Лучшее: 1.035 (f = -5.407)
  Разнообразие: 1.385
  Средняя точка кроссовера: 31.0

Поколение 70
  Среднее: 1.38
  Лучшее:

┌ Info: Saved animation to /Users/d.okutin/genetic_algorithm_binary_test2.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156


In [3]:
using Plots
using Random


f(x) = 5 - 24*x + 17*x^2 - (11/3)*x^3 + (2/9)*x^4


initial_pop_size = 50        
n_generations = 200   
x_range = (-3.0, 10.0) 
bits = 16   
tournament_size = 3
max_age = 20  


function int_to_bin(n::Int, bits::Int)
    s = ""
    for i in 1:bits
        power = 2^(bits - i)
        if n >= power
            s *= "1"
            n -= power
        else
            s *= "0"
        end
    end
    return s
end

function bin_to_int(bin_str::String)
    n = 0
    for ch in bin_str
        n = n * 2 + (ch == '1' ? 1 : 0)
    end
    return n
end

function float_to_bin(x::Float64, lower::Float64, upper::Float64, bits::Int)
    scale = (x - lower) / (upper - lower)
    max_int = 2^bits - 1
    n = round(Int, scale * max_int)
    return int_to_bin(n, bits)
end

function bin_to_float(bin_str::String, lower::Float64, upper::Float64, bits::Int)
    n = bin_to_int(bin_str)
    max_int = 2^bits - 1
    scale = n / max_int
    return lower + scale * (upper - lower)
end

function binary_crossover(parent1::Float64, parent2::Float64, lower::Float64, upper::Float64, bits::Int)
    bin1 = float_to_bin(parent1, lower, upper, bits)
    bin2 = float_to_bin(parent2, lower, upper, bits)
    
    cp = rand(1:bits-1)
    child_bin = bin1[1:cp] * bin2[cp+1:end]
    child = bin_to_float(child_bin, lower, upper, bits)
    return child
end


function tournament_selection(population, fitness_values, tournament_size)
    indices = rand(1:length(population), tournament_size)
    winner_idx = indices[argmin([fitness_values[i] for i in indices])]
    return population[winner_idx]
end


mutable struct Indiv
    value::Float64
    age::Int
    new_this_generation::Bool  
end



population = [Indiv(rand() * (x_range[2] - x_range[1]) + x_range[1], 0, false) for _ in 1:initial_pop_size]

best_x_per_gen = Float64[]
best_y_per_gen = Float64[]
pop_size_per_gen = Int[]


all_populations = []
push!(all_populations, deepcopy(population))

for generation in 1:n_generations
    
    
    for ind in population
        ind.age += 1
        ind.new_this_generation = false
    end
    
    
    values = [f(ind.value) for ind in population]
    
    
    best_index = argmin(values)
    best_x = population[best_index].value
    best_y = values[best_index]
    push!(best_x_per_gen, best_x)
    push!(best_y_per_gen, best_y)
    push!(pop_size_per_gen, length(population))
    
    
    filter!(ind -> ind.age < max_age, population)
    
    
    new_Indivs = []
    
    
    n_offspring = 5  
    
    for _ in 1:n_offspring
        
        values = [f(ind.value) for ind in population]
        parent1 = tournament_selection(population, values, tournament_size)
        parent2 = tournament_selection(population, values, tournament_size)
        
        
        if rand() < 0.7
            child_value = binary_crossover(parent1.value, parent2.value, x_range[1], x_range[2], bits)
        else
            
            child_value = rand(Bool) ? parent1.value : parent2.value
        end
        
        
        if rand() < 0.1
            child_value += randn() * 0.5
            
            child_value = max(x_range[1], min(x_range[2], child_value))
        end
        
        
        push!(new_Indivs, Indiv(child_value, 0, true))
    end
    
    
    append!(population, new_Indivs)
    
    

    
    push!(all_populations, deepcopy(population))
end


xs = range(-3, 10, length=400)
ys = [f(x) for x in xs]

anim = @animate for i in 1:n_generations
    current_pop = all_populations[i]
    
    
    p = plot(xs, ys,
        label="f(x)",
        title = "поколение $(i)",
        xlabel = "x",
        ylabel = "f(x)",
        legend = :topright,
        lw = 2,
        ylims = (-100, 400))  
    
    
    x_values = [ind.value for ind in current_pop]
    y_values = [f(ind.value) for ind in current_pop]
    
    
    for j in 1:length(current_pop)
        ind = current_pop[j]
        if !ind.new_this_generation
            
            remaining_lifetime = max_age - ind.age
            alpha = (remaining_lifetime / max_age)
            
            scatter!([ind.value], [f(ind.value)], 
                    color =:grey , alpha=alpha,
                    markersize = 6, 
                    label = (j == 1 ? "Существующий кандидат" : ""))
        end
    end
    
    
    new_inds = filter(ind -> ind.new_this_generation, current_pop)
    if !isempty(new_inds)
        new_x_values = [ind.value for ind in new_inds]
        new_y_values = [f(ind.value) for ind in new_inds]
        scatter!(new_x_values, new_y_values, 
                color = :red,
                markersize = 6, 
                label = "Новые кандидат")
    end
    
    
    scatter!([best_x_per_gen[i]], [best_y_per_gen[i]], 
            color = :blue,
            markersize = 8, 
            markershape = :star5,
            label = "Лучший кандидат")
    
    sleep(0.01)
    
end

gif(anim, "ga_age_based_all_candidates.gif", fps=10)

println("min: x = $(best_x_per_gen[end])")
println("f(x) = $(best_y_per_gen[end])")
println("fin popsize: $(length(population))")




min: x = 8.030609597924773
f(x) = -66.12479108165735
fin popsize: 100


┌ Info: Saved animation to /Users/d.okutin/ga_age_based_all_candidates.gif
└ @ Plots /Users/d.okutin/.julia/packages/Plots/kLeqV/src/animation.jl:156
